# Document Loading

## Note to students.
During periods of high load you may find the notebook unresponsive. It may appear to execute a cell, update the completion number in brackets [#] at the left of the cell but you may find the cell has not executed. This is particularly obvious on print statements when there is no output. If this happens, restart the kernel using the command under the Kernel tab.

## Retrieval augmented generation
In retrieval augmented generation (RAG), an LLM retrieves contextual documents from an external dataset as part of its execution.

This is useful if we want to ask question about specific documents (e.g., our PDFs, a set of videos, etc).

In [0]:
! pip install langchain

In [0]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

## PDFs
Let's load a PDF [transcript](https://see.stanford.edu/materials/aimlcs229/transcripts/MachineLearning-Lecture01.pdf) from Andrew Ng's famous CS229 course! These documents are the result of automated transcription so words and sentences are sometimes split unexpectedly.

In [0]:
# The course will show the pip installs you would need to install packages on your own machine.
# These packages are already installed on this platform and should not be run again.
# ! pip install pypdf 

In [0]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture01.pdf")
pages = loader.load()

Each page is a Document.

A Document contains text (page_content) and metadata.

In [0]:
len(pages)

22

In [0]:
page = pages[0]

In [0]:
print(page.page_content[0:500])

MachineLearning-Lecture01  
Instructor (Andrew Ng):  Okay. Good morning. Welcome to CS229, the machine 
learning class. So what I wanna do today is ju st spend a little time going over the logistics 
of the class, and then we'll start to  talk a bit about machine learning.  
By way of introduction, my name's  Andrew Ng and I'll be instru ctor for this class. And so 
I personally work in machine learning, and I' ve worked on it for about 15 years now, and 
I actually think that machine learning i


In [0]:
page.metadata

{'source': 'docs/cs229_lectures/MachineLearning-Lecture01.pdf', 'page': 0}

## YouTube

In [0]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

In [0]:
! pip install yt_dlp
! pip install pydub

     ---------------------------------------- 3.2/3.2 MB 14.4 MB/s eta 0:00:00
     ---------------------------------------- 357.3/357.3 kB ? eta 0:00:00
     ---------------------------------------- 64.9/64.9 kB ? eta 0:00:00
     ---------------------------------------- 159.1/159.1 kB ? eta 0:00:00
     ---------------------------------------- 126.3/126.3 kB ? eta 0:00:00
     ---------------------------------------- 194.4/194.4 kB ? eta 0:00:00
     ---------------------------------------- 1.8/1.8 MB 58.3 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.14
    Uninstalling urllib3-1.26.14:
      Successfully uninstalled urllib3-1.26.14
  Attempting uninstall: requests
    Found existing installation: requests 2.28.1
    Uninstalling requests-2.28.1:
      Successfully uninsta

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.41 requires requests_mock, which is not installed.
datasets 2.16.1 requires dill<0.3.8,>=0.3.0, but you have dill 0.3.8 which is incompatible.
conda-repo-cli 1.0.41 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.41 requires nbformat==5.4.0, but you have nbformat 5.7.0 which is incompatible.
conda-repo-cli 1.0.41 requires requests==2.28.1, but you have requests 2.32.3 which is incompatible.
botocore 1.34.37 requires urllib3<2.1,>=1.25.4; python_version >= "3.10", but you have urllib3 2.2.3 which is incompatible.


**Note**: This can take several minutes to complete.

In [0]:
url="https://www.youtube.com/watch?v=jGwO_UgTS7I"
save_dir="docs/youtube/"
loader = GenericLoader(
    YoutubeAudioLoader([url],save_dir),
    OpenAIWhisperParser()
)
docs = loader.load()

[youtube] Extracting URL: https://www.youtube.com/watch?v=jGwO_UgTS7I
[youtube] jGwO_UgTS7I: Downloading webpage
[youtube] jGwO_UgTS7I: Downloading ios player API JSON
[youtube] jGwO_UgTS7I: Downloading mweb player API JSON
[youtube] jGwO_UgTS7I: Downloading player 4e23410d
[youtube] jGwO_UgTS7I: Downloading m3u8 information
[info] jGwO_UgTS7I: Downloading 1 format(s): 140
[download] docs\youtube\Stanford CS229： Machine Learning Course, Lecture 1 - Andrew Ng (Autumn 2018).m4a has already been downloaded
[download] 100% of   69.71MiB


ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location


---------------------------------------------------------------------------
PostProcessingError                       Traceback (most recent call last)
File c:\Users\Sourav Banerjee\anaconda3\lib\site-packages\yt_dlp\YoutubeDL.py:3560, in YoutubeDL.process_info(self, info_dict)
   3559 try:
-> 3560     replace_info_dict(self.post_process(dl_filename, info_dict, files_to_move))
   3561 except PostProcessingError as err:

File c:\Users\Sourav Banerjee\anaconda3\lib\site-packages\yt_dlp\YoutubeDL.py:3744, in YoutubeDL.post_process(self, filename, info, files_to_move)
   3743 info['__files_to_move'] = files_to_move or {}
-> 3744 info = self.run_all_pps('post_process', info, additional_pps=info.get('__postprocessors'))
   3745 info = self.run_pp(MoveFilesAfterDownloadPP(self), info)

File c:\Users\Sourav Banerjee\anaconda3\lib\site-packages\yt_dlp\YoutubeDL.py:3726, in YoutubeDL.run_all_pps(self, key, info, additional_pps)
   3725 for pp in (additional_pps or []) + self._pps[key]:
-> 3726  

In [0]:
docs[0].page_content[0:500]

"Welcome to CS229 Machine Learning. Uh, some of you know that this is a class that's taught at Stanford for a long time. And this is often the class that, um, I most look forward to teaching each year because this is where we've helped, I think, several generations of Stanford students become experts in machine learning, got- built many of their products and services and startups that I'm sure, many of you or probably all of you are using, uh, uh, today. Um, so what I want to do today was spend s"

## URLs

In [0]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://github.com/basecamp/handbook/blob/master/titles-for-programmers.md")

In [0]:
docs = loader.load()

In [0]:
print(docs[0].page_content[:500])





















































































handbook/titles-for-programmers.md at master · basecamp/handbook · GitHub













































Skip to content













Navigation Menu

Toggle navigation




 













            Sign in
          








        Product
        













GitHub Copilot
        Write better code with AI
      







Security
        Find and fix vulnerabilities
      







Actions
      


## Notion

Follow steps [here](https://python.langchain.com/docs/modules/data_connection/document_loaders/integrations/notion) for an example Notion site such as [this one](https://yolospace.notion.site/Blendle-s-Employee-Handbook-e31bff7da17346ee99f531087d8b133f):

- Duplicate the page into your own Notion space and export as Markdown / CSV.
- Unzip it and save it as a folder that contains the markdown file for the Notion page.

In [0]:
from langchain.document_loaders import NotionDirectoryLoader
loader = NotionDirectoryLoader("docs/Notion_DB")
docs = loader.load()

In [0]:
print(docs[0].page_content[0:200])

# #letstalkaboutstress

Let’s talk about stress. Too much stress. 

We know this can be a topic.

So let’s get this conversation going. 

[Intro: two things you should know](#letstalkaboutstress%20640


In [0]:
docs[0].metadata

{'source': 'docs\\Notion_DB\\#letstalkaboutstress 64040a0733074994976118bbe0acc7fb.md'}